![](../images/30_translating_solveit_dialogues/thumbnail.png)

## Introduction

In this notebook I'll be demonstrating how one can update messages in SolveIt dialogues and do further work with them. At the same time also demonstrating how SolveIt dialogue messages can be translated with a LLM.

I'm currently teaching myself Chinese, and I've found to learn a language, one has to use the language. So I make the LLM respond in Chinese.

I'll be using [lisette](https://lisette.answer.ai/) for working with the LLM, DeepSeek as the model, and [dialoghelper](https://answerdotai.github.io/dialoghelper/) to process the dialogue messages.

## Sample Dialogue

I'll create a rough, short dialogue that I can work upon.

> You'll see that the English translations are already appended. See the SolveIt dialogue [here](https://share.solve.it.com/d/24bf30f133c01f37feae1a1e22e27bcc).

Hey there, who are you?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

你好！我是 solveit，一个学习助手。我可以帮你学习各种各样的东西——数学、编程、写作，或者别的什么。

你想学什么呢？你对什么感兴趣？告诉我一点儿关于你自己的情况吧，这样我可以更好地帮助你。

---

Hello! I'm solveit, a learning assistant. I can help you learn all sorts of things—math, programming, writing, or anything else.

What would you like to learn? What are you interested in? Tell me a little about yourself so I can help you better.

---

Hello! I'm solveit, a learning assistant. I can help you learn all sorts of things—math, programming, writing, or anything else.

What would you like to learn? What are you interested in? Tell me a little about yourself so I can help you better.

Let me flip it around: what do _you_ want to study? What're _you_ currently interested in?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

哈哈，这个问题很有意思！

其实我对很多东西都感兴趣。最近我特别喜欢思考这些问题：

- 人们怎么学习新东西？什么方法最有效？
- 复杂的问题怎么分解成简单的部分？
- 不同领域的知识怎么连接起来？

但是，我在这儿是为了帮助你学习和探索。你的好奇心和兴趣才是最重要的！

所以，咱们可以一起探索一个你感兴趣的话题。你现在想学什么？或者你正在做什么项目？

---

Haha, that's a really interesting question!

Actually, I'm interested in a lot of things. Lately, I've been particularly fond of thinking about these questions:

- How do people learn new things? What methods are most effective?
- How can complex problems be broken down into simpler parts?
- How is knowledge from different fields connected?

However, I'm here to help *you* learn and explore. Your curiosity and interests are what matter most!

So, we can explore a topic you're interested in together. What would you like to learn right now? Or are you working on any projects?

## Translation

In [ ]:
from dialoghelper import *

In [ ]:
?find_msgs

Signature:
find_msgs(
    re_pattern: str = '',
    msg_type: str = None,
    limit: int = None,
    include_output: bool = True,
    dname: str = '',
)
Docstring: Find `list[dict]` of messages in current specific dialog that contain the given information. To refer to a message found later, use its `id` field.
File:      /usr/local/lib/python3.12/site-packages/dialoghelper/core.py
Type:      function

We can use `find_msgs` to filter out those message types that are prompts.

In [ ]:
o = find_msgs(msg_type='prompt'); o[0]

{'id': '_4dcda3fa',
 'input_tokens': 10,
 'output_tokens': 94,
 'time_run': '',
 'is_exported': False,
 'skipped': False,
 'hidden': False,
 'bookmark': None,
 'i_collapsed': False,
 'o_collapsed': False,
 'heading_collapsed': False,
 'i_clamp': False,
 'o_clamp': False,
 'pinned': False,
 'use_thinking': True,
 'attachments': [],
 'oob': None,
 'run': None,
 'scroll': False,
 'order': 3,
 'content': 'Hey there, who are you?',
 'output': '🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠\n\n你好！我是 solveit，一个学习助手。我可以帮你学习各种各样的东西——数学、编程、写作，或者别的什么。\n\n你想学什么呢？你对什么感兴趣？告诉我一点儿关于你自己的情况吧，这样我可以更好地帮助你。',
 'msg_type': 'prompt'}

In [ ]:
o[0]['output']

'🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠\n\n你好！我是 solveit，一个学习助手。我可以帮你学习各种各样的东西——数学、编程、写作，或者别的什么。\n\n你想学什么呢？你对什么感兴趣？告诉我一点儿关于你自己的情况吧，这样我可以更好地帮助你。'

And then when we are iterating through all the prompt messages we can fetch the respective message ID and update it using `update_msg`.

In [ ]:
?update_msg

Signature:
update_msg(
    msgid: str = None,
    msg: Optional[Dict] = None,
    dname: str = '',
    *,
    content: str | None = None,
    msg_type: str | None = None,
    output: str | None = None,
    time_run: str | None = None,
    is_exported: int | None = None,
    skipped: int | None = None,
    i_collapsed: int | None = None,
    o_collapsed: int | None = None,
    heading_collapsed: int | None = None,
    pinned: int | None = None,
)
Docstring:
Update an existing message. Provide either `msg` OR field key/values to update.
- Use `content` param to update contents.
- Only include parameters to update--missing ones will be left unchanged.
File:      /usr/local/lib/python3.12/site-packages/dialoghelper/core.py
Type:      function

I'll be formatting the new updated prompt output as `original + '\n\n---\n\n' + translated`.

In a nutshell, the core loop is:

1. Fetch a prompt message.
2. Access the output content of that message.
3. Translate.
4. Reformat output content
5. Update the respective message.

I'll begin with a single example.

In [ ]:
from lisette.core import *
sys_prmpt = "You'll receive a series of messages that are responses in a dialogue. I need you to aptly translate those responses to English, whilst maintaining the same tone/style and minimal loss of information. If you encoutner any brain emojies, skip them. Do not re-output the original message."
chat = Chat('deepseek/deepseek-chat', hist=[sys_prmpt]); chat

In [ ]:
res = chat(o[0]['output'])
display(res)

Hello! I'm solveit, a learning assistant. I can help you learn all sorts of things—math, programming, writing, or anything else.

What would you like to learn? What are you interested in? Tell me a little about yourself so I can help you better.

<details>

- id: `22a761da-f8a1-43a4-be6f-06a54db8ca0f`
- model: `deepseek-chat`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=57, prompt_tokens=155, total_tokens=212, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=64, text_tokens=None, image_tokens=None), prompt_cache_hit_tokens=64, prompt_cache_miss_tokens=91)`

</details>

In [ ]:
display(chat(o[1]['output']))

Haha, that's a very interesting question!

Actually, I'm interested in a lot of things. Lately, I've been particularly fond of pondering these questions:

- How do people learn new things? What methods are most effective?
- How can complex problems be broken down into simpler parts?
- How is knowledge from different fields connected?

However, I'm here to help *you* learn and explore. Your curiosity and interests are what matter most!

So, we can explore a topic you're interested in together. What would you like to learn right now? Or are you working on any projects?

<details>

- id: `cb9c2af2-e7ed-48cf-ae27-dbf56bd1684c`
- model: `deepseek-chat`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=123, prompt_tokens=300, total_tokens=423, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=128, text_tokens=None, image_tokens=None), prompt_cache_hit_tokens=128, prompt_cache_miss_tokens=172)`

</details>

So that's how the translations look. Let's now update the messages themselves. 

In [ ]:
??update_msg


```python
@delegates(_umsg)
def update_msg(
    msgid:str=None, # id of message to update (if None, uses current message)
    msg:Optional[Dict]=None, # Dictionary of field keys/values to update
    dname:str='', # Running dialog to get info for; defaults to current dialog
    **kwargs):
    """Update an existing message. Provide either `msg` OR field key/values to update.
    - Use `content` param to update contents.
    - Only include parameters to update--missing ones will be left unchanged."""
    if not msgid and not msg: raise TypeError("update_msg needs either a dict message or `msgid=`")
    res = call_endp('update_msg_', dname, msgid=msgid, **kwargs)
    set_var('__msg_id', res)
    return res
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`

In [ ]:
trans = res.choices[0].message.content; trans

"Hello! I'm solveit, a learning assistant. I can help you learn all sorts of things—math, programming, writing, or anything else.\n\nWhat would you like to learn? What are you interested in? Tell me a little about yourself so I can help you better."

In [ ]:
new_out = o[0]['output']+'\n\n---\n\n'+trans; new_out

"🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠\n\n你好！我是 solveit，一个学习助手。我可以帮你学习各种各样的东西——数学、编程、写作，或者别的什么。\n\n你想学什么呢？你对什么感兴趣？告诉我一点儿关于你自己的情况吧，这样我可以更好地帮助你。\n\n---\n\nHello! I'm solveit, a learning assistant. I can help you learn all sorts of things—math, programming, writing, or anything else.\n\nWhat would you like to learn? What are you interested in? Tell me a little about yourself so I can help you better."

In [ ]:
update_msg(msgid=o[0]['id'], output=new_out)

'_4dcda3fa'

In [ ]:
o[0]

{'id': '_4dcda3fa',
 'input_tokens': 10,
 'output_tokens': 94,
 'time_run': '',
 'is_exported': False,
 'skipped': False,
 'hidden': False,
 'bookmark': None,
 'i_collapsed': False,
 'o_collapsed': False,
 'heading_collapsed': False,
 'i_clamp': False,
 'o_clamp': False,
 'pinned': False,
 'use_thinking': True,
 'attachments': [],
 'oob': None,
 'run': None,
 'scroll': False,
 'order': 3,
 'content': 'Hey there, who are you?',
 'output': '🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠\n\n你好！我是 solveit，一个学习助手。我可以帮你学习各种各样的东西——数学、编程、写作，或者别的什么。\n\n你想学什么呢？你对什么感兴趣？告诉我一点儿关于你自己的情况吧，这样我可以更好地帮助你。',
 'msg_type': 'prompt'}

The message has been updated! <a href="javascript:document.getElementById('_4dcda3fa').scrollIntoView({behavior: 'smooth'})">Click here</a> to jump to it and see the update.

Now we can package everything up into a loop. 

In [ ]:
chat = Chat('deepseek/deepseek-chat', hist=[sys_prmpt])
outs = find_msgs(msg_type='prompt')
for o in outs:
    r = chat(o['output'])
    trans = r.choices[0].message.content
    new_o = o['output']+'\n\n---\n\n'+trans
    update_msg(msgid=o['id'], output=new_o)

Some areas for improvement:
- [ ] The reformatted output includes collapsable/folding blocks
- [ ] Figure out how to use [fastprogress](https://github.com/AnswerDotAI/fastprogress) view translation progress.

## Conclusion

And it's as simple as that! 

If you have any comments, questions, suggestions, feedback, criticisms, or corrections, please do post them down in the comment section below!